TODO
1. extract features by googlenet
2. compute similarity
3. test on Place dataset

In [23]:
# load places205 dataset
import hub
ds = hub.load("hub://activeloop/places205")
dataloader = ds.pytorch(num_workers = 2, shuffle = False, batch_size= 4)

# import numpy as np
# img = ds.images[42].numpy()
# print(img.shape)

# import matplotlib.pyplot as plt
# plt.imshow(img)

Opening dataset in read-only mode as you don't have write permissions.
hub://activeloop/places205 loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/activeloop/places205.


In [52]:
import torch
import torch.nn as nn
from googlenet_places205 import GoogLeNetPlaces205

model = GoogLeNetPlaces205()
model.load_state_dict(torch.load("googlenet_places205.pth"))

# define hook
features = {}
def get_middle_output(name):
    def hook(model, input, output):
        features[name] = (output.detach())
    return hook

# model.inception_3b_5x5.register_forward_hook(get_middle_output('icp1'))
model.inception_4b_5x5.register_forward_hook(get_middle_output('icp1'))

In [53]:
from PIL import Image
from torchvision import transforms

def preprocess(filename):
    input_image = Image.open(filename)
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = transform(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    return input_batch

def get_features(input):
    output = model(input)
    return features['icp1'].flatten()

filename = ["images/road.png", "images/road2.png", "images/rail.png", "images/rail2.png"]
outputs = []
for img in filename:
    input = preprocess(img)
    outputs.append(get_features(input))

In [54]:
def get_similarity(input1, input2):
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    return cos(input1, input2)

print(get_similarity(outputs[0], outputs[1]))
print(get_similarity(outputs[0], outputs[2]))
print(get_similarity(outputs[0], outputs[3]))

tensor(0.9086)
tensor(0.8983)
tensor(0.9108)
